# Scenario 1 : linear case

## library

In [1]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects import numpy2ri
from rpy2.robjects.vectors import StrVector

required_packages = ['base', 'forecast', 'CondIndTests','RCIT'] # list of required R packages 
if all(rpackages.isinstalled(x) for x in required_packages):
    check_packages = True # True if packages are already installed 
else:
   check_packages = False # False if packages are not installed 
if check_packages == False: # Not installed? Then install.
    utils = rpackages.importr('utils')
    utils.chooseCRANmirror(ind=1)
    packages_to_install = [x for x in required_packages if not rpackages.isinstalled(x)]
    if len(packages_to_install) > 0:
        utils.install_packages(StrVector(packages_to_install))
    check_packages = True 

In [2]:
r = robjects.r
base = importr('base')
forecast = importr('forecast')
graphics = importr('graphics')
grdevices = importr('grDevices')
condindtests = importr('CondIndTests')

In [3]:
RCIT = importr('RCIT')

In [4]:
pcalg = importr('pcalg')

In [5]:
def IgnorTest(cause=None, effect=None, explorer=None, covariate=None, level=0.05, dtype="continuous"): #inputs are arrays, dtypes are continuous/binary/mixed
    tmp_cause=pd.DataFrame({'A': cause})
    tmp_effect=pd.DataFrame({'Y': effect})
    tmp_explorer=pd.DataFrame({'V': explorer})
    tmp_covariate = pd.DataFrame(covariate)
    dim_covariate = tmp_covariate.shape[1]
    
    if dtype=="continuous":
        if dim_covariate==0:
            tmp_conditioning=tmp_cause
        else:
            tmp_conditioning=pd.concat([tmp_cause,tmp_covariate],axis=1)

        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_conditioning = ro.conversion.py2rpy(tmp_conditioning)
        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_effect = ro.conversion.py2rpy(tmp_effect)
        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_explorer = ro.conversion.py2rpy(tmp_explorer)
            
        sample_size = r.unlist(r.dim(r_from_pd_conditioning))[0]
        dim = r.unlist(r.dim(r_from_pd_conditioning))[1]
            
        r_from_pd_explorer=r.unlist(r_from_pd_explorer)    
        r_from_pd_effect=r.unlist(r_from_pd_effect)   
        r_from_pd_conditioning=r.unlist(r_from_pd_conditioning)   
        r_from_pd_conditioning=r.matrix(r_from_pd_conditioning,sample_size,dim)
        
        #test=r.CondIndTest(r_from_pd_explorer,r_from_pd_effect,r_from_pd_conditioning, method = "KCI")
        #pvalue=r.unlist(test)[3]
        
        test=r.RCoT(r_from_pd_explorer,r_from_pd_effect,r_from_pd_conditioning,seed=2)
        pvalue=r.unlist(test)[0]
        print(pvalue)
           
    elif dtype=="binary":
        if covariate==None:
            tmp_conditioning=tmp_cause
        else:
            tmp_covariate=pd.DataFrame(covariate)
            tmp_conditioning=pd.concat([tmp_cause,tmp_covariate],axis=1)
            
    elif dtype=="mixed":
        pass
                        
    else:
        raise ValueError("explorertype is not valid.")
        
    if(pvalue<=level):
        print("The Ignorability Assumption is not satisfied.")
    else:
        print("The Ignorability Assumption is satisfied.")

    return pvalue

### parameter setting

In [6]:
n = 4000 #changeable
c1 = 1
c2 = 1 
c3 = 1 #changeable
c4 = 1
c5 = 1
r1 = r2 = r3 = r4 = 1 #changeable
import scipy.stats as stats

### Simulation

In [7]:
#Figure 1
np.random.seed(1)

COUNT1=0 #기각 횟수
for i in range(1000):

    U = np.random.normal(0,1,n) #common cause of X1 and X2
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+e1
    X2 = r2*U+e2
    Y = c1*X1+ey
    
    if i==1:
        f1_cor12=stats.pearsonr(X1, X2)[0]
    
    pvalue = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous")
    if pvalue<=0.05:
        COUNT1 = COUNT1+1

0.5194749570558681
The Ignorability Assumption is satisfied.
0.7305534066474422
The Ignorability Assumption is satisfied.
0.7743015216513794
The Ignorability Assumption is satisfied.
0.32462364890490436
The Ignorability Assumption is satisfied.
0.5419141508836413
The Ignorability Assumption is satisfied.
0.40578300140438517
The Ignorability Assumption is satisfied.
0.3704775139569989
The Ignorability Assumption is satisfied.
0.8132293725375003
The Ignorability Assumption is satisfied.
0.6600290594555094
The Ignorability Assumption is satisfied.
0.23300096859787234
The Ignorability Assumption is satisfied.
0.7883776428834083
The Ignorability Assumption is satisfied.
0.1615157817578884
The Ignorability Assumption is satisfied.
0.8015420254860322
The Ignorability Assumption is satisfied.
0.3732813333990509
The Ignorability Assumption is satisfied.
0.7383474339713717
The Ignorability Assumption is satisfied.
0.055649094646017305
The Ignorability Assumption is satisfied.
0.01100510161135204

In [8]:
#Figure 2
np.random.seed(1)

COUNT21=0 #기각 횟수
COUNT22=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    Y = c1*X1+c3*C+ey
    
    if i==1:
        f2_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT21 = COUNT21+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT22 = COUNT22+1 #2종 오류

0.9054324932938418
The Ignorability Assumption is satisfied.
4.440892098500626e-16
The Ignorability Assumption is not satisfied.
0.7633969710923181
The Ignorability Assumption is satisfied.
8.881784197001252e-16
The Ignorability Assumption is not satisfied.
0.5995364439965325
The Ignorability Assumption is satisfied.
1.5543122344752192e-15
The Ignorability Assumption is not satisfied.
0.8124584393293008
The Ignorability Assumption is satisfied.
6.661338147750939e-16
The Ignorability Assumption is not satisfied.
0.4166726667768921
The Ignorability Assumption is satisfied.
7.771561172376096e-16
The Ignorability Assumption is not satisfied.
0.31664345113344894
The Ignorability Assumption is satisfied.
5.551115123125783e-16
The Ignorability Assumption is not satisfied.
0.9906223468900821
The Ignorability Assumption is satisfied.
1.1102230246251565e-16
The Ignorability Assumption is not satisfied.
0.901915405703182
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is

In [9]:
COUNT21

63

In [10]:
np.max(X2)

6.5844033590288085

In [11]:
#Figure 3
np.random.seed(1)

COUNT31=0 #기각 횟수
COUNT32=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    Y = c3*C+ey
    
    if i==1:
        f3_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT31 = COUNT31+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT32 = COUNT32+1 #2종 오류

0.8287589781666992
The Ignorability Assumption is satisfied.
2.220446049250313e-16
The Ignorability Assumption is not satisfied.
0.7816890512892364
The Ignorability Assumption is satisfied.
3.3306690738754696e-16
The Ignorability Assumption is not satisfied.
0.37402332998179
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.6188380603169426
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.17917061705044224
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.6113681290900463
The Ignorability Assumption is satisfied.
1.4432899320127035e-15
The Ignorability Assumption is not satisfied.
0.9047963592038256
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.9097996136499213
The Ignorability Assumption is satisfied.
2.55351295663786e-15
The Ignorability Assumption is not satisfied.
0.8751090756869818
The Ignorability Assu

In [12]:
#Figure 4 
np.random.seed(1)

COUNT41=0 #기각 횟수
COUNT42=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    X3 = r3*U+e3
    Y = c1*X1+c4*X2+c3*C+ey
    
    if i==1:
        f4_cor13=stats.pearsonr(X1, X3)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=np.column_stack((X2,C)),dtype="continuous") 
    if pvalue1<=0.05:
        COUNT41 = COUNT41+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT42 = COUNT42+1 #2종 오류

0.442321789236815
The Ignorability Assumption is satisfied.
4.440892098500626e-16
The Ignorability Assumption is not satisfied.
0.8928755185739484
The Ignorability Assumption is satisfied.
1.7087442572005784e-12
The Ignorability Assumption is not satisfied.
0.4257541421486597
The Ignorability Assumption is satisfied.
2.0643825979771435e-06
The Ignorability Assumption is not satisfied.
0.6220336663869093
The Ignorability Assumption is satisfied.
3.9968028886505635e-15
The Ignorability Assumption is not satisfied.
0.8805737505448392
The Ignorability Assumption is satisfied.
2.48245868306185e-13
The Ignorability Assumption is not satisfied.
0.6365265679692674
The Ignorability Assumption is satisfied.
1.887379141862766e-15
The Ignorability Assumption is not satisfied.
0.775566210370134
The Ignorability Assumption is satisfied.
1.817809514026436e-08
The Ignorability Assumption is not satisfied.
0.6188129806125708
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is n

In [13]:
#Figure 5
np.random.seed(1)

COUNT51=0 #기각 횟수
COUNT52=0
COUNT53=0
COUNT54=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    X3 = r3*U+e3
    Y = c1*X1+c3*C+ey
    
    if i==1:
        f5_cor12=r1*r2/np.sqrt(r1**2+c2**2+1)/np.sqrt(r2**2+1)
        f5_cor13=r1*r3/np.sqrt(r1**2+c2**2+1)/np.sqrt(r3**2+1)
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT51 = COUNT51+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT52 = COUNT52+1 #2종 오류
    pvalue3 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=C,dtype="continuous") 
    if pvalue3<=0.05:
        COUNT53 = COUNT53+1 #1종 오류
    pvalue4 = IgnorTest(cause=X1,effect=Y,explorer=X3,dtype="continuous") 
    if pvalue4<=0.05:
        COUNT54 = COUNT54+1 #2종 오류

0.04624165495699284
The Ignorability Assumption is not satisfied.
3.3306690738754696e-16
The Ignorability Assumption is not satisfied.
0.8939043822332537
The Ignorability Assumption is satisfied.
1.1102230246251565e-16
The Ignorability Assumption is not satisfied.
0.08734567619082423
The Ignorability Assumption is satisfied.
3.3306690738754696e-16
The Ignorability Assumption is not satisfied.
0.8014715545957818
The Ignorability Assumption is satisfied.
2.220446049250313e-16
The Ignorability Assumption is not satisfied.
0.40279543424200326
The Ignorability Assumption is satisfied.
4.440892098500626e-16
The Ignorability Assumption is not satisfied.
0.592594712466354
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.3662462748730979
The Ignorability Assumption is satisfied.
1.7763568394002505e-15
The Ignorability Assumption is not satisfied.
0.6692087090236234
The Ignorability Assumption is satisfied.
1.2212453270876722e-15
The Ignorability Assu

In [14]:
print(f5_cor12)
print(f5_cor13)

0.408248290463863
0.408248290463863


In [15]:
COUNT51

56

In [16]:
COUNT52

1000

In [17]:
COUNT53

61

In [18]:
COUNT54

1000

In [19]:
#r_y2 = stats.pearsonr(Y, X2)[0]
#r_y1 = stats.pearsonr(Y, X1)[0]
#r_12 = stats.pearsonr(X1, X2)[0]

In [20]:
#Figure 6
np.random.seed(1)

COUNT61=0 #기각 횟수
COUNT62=0
COUNT63=0
COUNT64=0
for i in range(1000):
    U1 = np.random.normal(0,1,n) #common cause of X1 and X2
    U2 = np.random.normal(0,1,n) #common cause of X1 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U1+c2*C+r3*U2+e1
    X2 = r2*U1+e2
    X3 = r4*U2+e3
    Y = c1*X1+c3*C+ey
    
    if i==1:
        f6_cor12=r1*r2/np.sqrt(r1**2+c2**2+r3**2+1)/np.sqrt(r2**2+1)
        f6_cor13=r3*r4/np.sqrt(r1**2+c2**2+r3**2+1)/np.sqrt(r4**2+1)
        
        r_y2 = stats.pearsonr(Y, X2)[0]
        r_y1 = stats.pearsonr(Y, X1)[0]
        r_12 = stats.pearsonr(X1, X2)[0]
        s2 = np.sqrt(r2**2+1)
        sy = np.sqrt(c1**2*(r1**2+c2**2+r3**2+1)+c3**2+1)
        
        f6_par_cor = (r_y2-r_y1*r_12)/np.sqrt(1-r_y1**2)/np.sqrt(1-r_12**2)

    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT61 = COUNT61+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT62 = COUNT62+1 #2종 오류
    pvalue3 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=C,dtype="continuous") 
    if pvalue3<=0.05:
        COUNT63 = COUNT63+1 #1종 오류
    pvalue4 = IgnorTest(cause=X1,effect=Y,explorer=X3,dtype="continuous") 
    if pvalue4<=0.05:
        COUNT64 = COUNT64+1 #2종 오류

0.46997772264949156
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.3083561207237038
The Ignorability Assumption is satisfied.
5.218048215738236e-15
The Ignorability Assumption is not satisfied.
0.4453883328507092
The Ignorability Assumption is satisfied.
1.1547429679126253e-12
The Ignorability Assumption is not satisfied.
0.5818607640999421
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.9972867885106902
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.2554741376188828
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.5542697153596043
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.7391872253730554
The Ignorability Assumption is satisfied.
4.773959005888173e-15
The Ignorability Assumption is not satisfied.
0.3251035152538224
The Ignorability Assumption is satisf

In [21]:
c3

1

In [22]:
f6_par_cor

-0.1297340361362299

In [23]:
print(f6_cor12)
print(f6_cor13)

0.35355339059327373
0.35355339059327373


In [24]:
COUNT61

41

In [25]:
COUNT62

1000

In [26]:
COUNT63

52

In [27]:
COUNT64

1000

In [28]:
#Figure 7
np.random.seed(1)

COUNT71=0 #기각 횟수
COUNT72=0
for i in range(1000):
    C = np.random.normal(0,1,n) #Confounder
    eu = np.random.normal(0,1,n)
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    U = c2*C+eu #common cause of X1 and X2
    X1 = r1*U+e1
    X2 = r2*U+e2
    Y = c1*X1+c3*C+ey
    
    if i==1:
        f7_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT71 = COUNT71+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT72 = COUNT72+1 #2종 오류

0.6835097651909261
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.6997449463379736
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.391162855791644
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.917638640747381
The Ignorability Assumption is satisfied.
1.4432899320127035e-15
The Ignorability Assumption is not satisfied.
0.06668079156383477
The Ignorability Assumption is satisfied.
6.661338147750939e-16
The Ignorability Assumption is not satisfied.
0.3518613746458644
The Ignorability Assumption is satisfied.
7.771561172376096e-16
The Ignorability Assumption is not satisfied.
0.9495238426506869
The Ignorability Assumption is satisfied.
1.1102230246251565e-16
The Ignorability Assumption is not satisfied.
0.17925345898555278
The Ignorability Assumption is satisfied.
2.220446049250313e-16
The Ignorability Assumption is not satisfied.
0.5966814925317183
T

In [29]:
#Figure 8
np.random.seed(1)

COUNT81=0 #기각 횟수
COUNT82=0
for i in range(1000):
    C = np.random.normal(0,1,n) #Confounder, common cause of X1 and X2
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*C+e1
    X2 = r2*C+e2
    Y = c1*X1+c3*C+ey
    
    if i==1:
        f8_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT81 = COUNT81+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT82 = COUNT82+1 #2종 오류

0.21394996855280035
The Ignorability Assumption is satisfied.
1.5543122344752192e-15
The Ignorability Assumption is not satisfied.
0.4601388700284481
The Ignorability Assumption is satisfied.
3.219646771412954e-15
The Ignorability Assumption is not satisfied.
0.8312933577992051
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.3705993943484165
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.7892632005245157
The Ignorability Assumption is satisfied.
3.3306690738754696e-16
The Ignorability Assumption is not satisfied.
0.8615370784745411
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.43251328779227494
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.9456360896728958
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.8938087083432696
The Ignorability Assumption is sati

In [30]:
#Figure 9
np.random.seed(1)

COUNT91=0 #기각 횟수
COUNT92=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+c4*C+e2
    Y = c1*X1+c3*C+ey
    
    if i==1:
        f9_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT91 = COUNT91+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT92 = COUNT92+1 #2종 오류

0.9127835928903425
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.8921010734343645
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.7061137110598082
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.8461390431131721
The Ignorability Assumption is satisfied.
2.9976021664879227e-15
The Ignorability Assumption is not satisfied.
0.3203125246387015
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.6266383819705701
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.9913558884789778
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.9404881275832768
The Ignorability Assumption is satisfied.
5.551115123125783e-16
The Ignorability Assumption is not satisfied.
0.8757864234444585
The Ignorability Assumption is satisfied.
0.0
The Ignora

In [31]:
#Figure 10
np.random.seed(1)

COUNT101=0 #기각 횟수
COUNT102=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+c4*C+e2
    X3 = r3*U+e3
    Y = c1*X1+c3*C+c5*X2+ey
    
    if i==1:
        f10_cor13=stats.pearsonr(X1, X3)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=np.column_stack((X2,C)),dtype="continuous") 
    if pvalue1<=0.05:
        COUNT101 = COUNT101+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT102 = COUNT102+1 #2종 오류

0.45037237863730206
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.8321907635178281
The Ignorability Assumption is satisfied.
1.5543122344752192e-15
The Ignorability Assumption is not satisfied.
0.32443444047101444
The Ignorability Assumption is satisfied.
7.771561172376096e-16
The Ignorability Assumption is not satisfied.
0.41737082202731013
The Ignorability Assumption is satisfied.
4.440892098500626e-16
The Ignorability Assumption is not satisfied.
0.8528974455167029
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.5752267546281671
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.8070121411226234
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.631373751001623
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.36371810518351844
The Ignorability Assumption is sati

In [32]:
table1=pd.DataFrame(np.repeat(-1,70).reshape(10,7))
table1.columns = ["Scenario1","1종오류(X2)","2종오류(X2)","1종오류(X3)","2종오류(X3)","r12","r13"]
table1

,Scenario1,1종오류(X2),2종오류(X2),1종오류(X3),2종오류(X3),r12,r13
0,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1
5,-1,-1,-1,-1,-1,-1,-1
6,-1,-1,-1,-1,-1,-1,-1
7,-1,-1,-1,-1,-1,-1,-1
8,-1,-1,-1,-1,-1,-1,-1
9,-1,-1,-1,-1,-1,-1,-1


In [33]:
table1["Scenario1"]=["Figure"+str(i+1) for i in range(10)]

In [34]:
table1["1종오류(X2)"]=[COUNT1,COUNT21,COUNT31,COUNT41,COUNT51,COUNT61,COUNT71,COUNT81,COUNT91,COUNT101]
table1["2종오류(X2)"]=[-1,COUNT22,COUNT32,COUNT42,COUNT52,COUNT62,COUNT72,COUNT82,COUNT92,COUNT102]
table1["1종오류(X3)"]=[-1,-1,-1,-1,COUNT53,COUNT63,-1,-1,-1,-1]
table1["2종오류(X3)"]=[-1,-1,-1,-1,COUNT54,COUNT64,-1,-1,-1,-1]
table1["r12"]=[f1_cor12,f2_cor12,f3_cor12,-1,f5_cor12,f6_cor12,f7_cor12,f8_cor12,f9_cor12,-1]
table1["r13"]=[-1,-1,-1,f4_cor13,f5_cor13,f6_cor13,-1,-1,-1,f10_cor13]

In [35]:
table1

,Scenario1,1종오류(X2),2종오류(X2),1종오류(X3),2종오류(X3),r12,r13
0,Figure1,53,-1,-1,-1,0.499098,-1.000000
1,Figure2,63,1000,-1,-1,0.413865,-1.000000
2,Figure3,53,1000,-1,-1,0.413865,-1.000000
3,Figure4,65,1000,-1,-1,-1.000000,0.396688
4,Figure5,56,1000,61,1000,0.408248,0.408248
5,Figure6,41,1000,52,1000,0.353553,0.353553
6,Figure7,61,1000,-1,-1,0.664412,-1.000000
7,Figure8,47,1000,-1,-1,0.499098,-1.000000
8,Figure9,67,1000,-1,-1,0.664412,-1.000000
9,Figure10,59,1000,-1,-1,-1.000000,0.396688


In [36]:
filename="LINEAR_c"+str(c3)+"_r"+str(r1)+"_s"+str(n/1000)+".csv"
table1.to_csv("C:\\Users\\minsoo\\Desktop\\실험결과(P)\\"+filename,encoding="euc-kr",index=False)